<a href="https://colab.research.google.com/github/sheneman/shakeGPT/blob/main/llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title shakeGPT - A Transformer Model
###########################################################
#
# ShakeGPT
#
# Learning to emit endless Shakespeare to demonstrate how
# decoder-only next-token prediction transformer models
# work
#
#
# Luke Sheneman
# sheneman@uidaho.edu
# December 2023
#
# Research Computing and Data Services (RCDS)
# Insitite for Interdisciplinary Data Sciences (IIDS)
# University of Idaho
#
##########################################################


Here we import our Python libraries

In [ ]:
import torch
from torch.nn import functional as F
import torch.nn as nn
import time
import requests

Here we define a number of hyperparameters which indicate where to find our training corpus and how to train our transformer:

In [45]:
INPUT_CORPUS     = "complete_shakespeare.txt"
INPUT_URL        = "https://www.gutenberg.org/ebooks/100.txt.utf-8"
OUTPUT_FILE      = "AI_shakespeare_out.txt"

SEED             = int(time.time())
TRAIN_SIZE       = 0.85
VAL_SIZE         = 1.0 - TRAIN_SIZE
BATCH_SIZE       = 200
NUM_BATCHES      = 2000
LEARNING_RATE    = 3e-4
CONTEXT_WINDOW   = 32
EMBEDDING_SIZE   = 64
NUM_LAYERS       = 6
NUM_HEADS        = 8
FFN_HIDDEN_SIZE  = 8
LOSS_SAMPLE_SIZE = 50

Some fabulous ASCII art.   Because.

In [37]:
shakespeare = """
   ⠀⠀⠀⠀⠀⠀⠀⠀⣀⣀⣠⣄⣀⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀
   ⠀⠀⠀⠀⠀⢀⣴⠟⠛⠉⠉⠉⠉⠛⠻⣦⡀⠀⠀⠀⠀⠀⠀
   ⠀⠀⠀⠀⢰⣿⠁⠀⠀⠀⠀⠀⠀⠀⠀⠈⣿⡆⠀⠀⠀⠀⠀
   ⠀⠀⠀⠀⣼⣿⣦⠀⠀⠀⠀⠀⠀⠀⠀⣴⣿⣷⡀⠀⠀⠀⠀
   ⠀⠀⠀⣰⣿⣿⣿⣤⣤⣄⠀⠀⣠⣤⣤⣿⣿⣿⣷⡀⠀⠀⠀
   ⠀⢀⣼⣿⣿⣿⠋⢠⣤⠙⠁⠈⠋⣤⡄⠙⣿⣿⣿⣿⣄⠀⠀
   ⢠⣿⣿⣿⣿⡿⠀⠈⠉⠀⠀⠀⠀⠉⠁⠀⢿⣿⣿⣿⣿⣷⠀
   ⣿⣿⣿⣿⣿⣇⠀⠀⠀⠀⡀⢀⠀⠀⠀⠀⣸⣿⣿⣿⣿⣿⡆
   ⠹⣿⣿⣿⣿⣿⠀⠀⠴⠞⠁⠈⠳⠦⠀⠀⣿⣿⣿⣿⣿⡿⠁
   ⠀⠉⢻⡿⢿⣿⣧⠀⠀⠀⢶⡶⠀⠀⠀⣼⣿⣿⣿⡟⠋⠁⠀
   ⠀⠀⣼⡇⠀⠀⠙⣷⣄⠀⠈⠁⠀⣠⣾⠋⠀⠀⢸⣧⠀⠀⠀
   ⠀⠀⣿⡇⠀⠀⠀⠈⠛⠷⣶⣶⠾⠛⠁⠀⠀⠀⢸⣿⠀⠀⠀
   ⠀⠀⢻⡇⠀⠀⠀⣀⣀⣤⣤⣤⣤⣀⣀⠀⠀⠀⢸⡟⠀⠀⠀
   ⠀⠀⠘⣿⣴⠾⠛⠋⠉⠉⠉⠉⠉⠉⠛⠛⠷⣦⣿⠃⠀⠀⠀
   ⠀⠀⠀⠈⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠁⠀⠀⠀⠀
All your word are belong to us!

     Shakespeare AI @ UI

"""

print(shakespeare)



   ⠀⠀⠀⠀⠀⠀⠀⠀⣀⣀⣠⣄⣀⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀
   ⠀⠀⠀⠀⠀⢀⣴⠟⠛⠉⠉⠉⠉⠛⠻⣦⡀⠀⠀⠀⠀⠀⠀
   ⠀⠀⠀⠀⢰⣿⠁⠀⠀⠀⠀⠀⠀⠀⠀⠈⣿⡆⠀⠀⠀⠀⠀
   ⠀⠀⠀⠀⣼⣿⣦⠀⠀⠀⠀⠀⠀⠀⠀⣴⣿⣷⡀⠀⠀⠀⠀
   ⠀⠀⠀⣰⣿⣿⣿⣤⣤⣄⠀⠀⣠⣤⣤⣿⣿⣿⣷⡀⠀⠀⠀
   ⠀⢀⣼⣿⣿⣿⠋⢠⣤⠙⠁⠈⠋⣤⡄⠙⣿⣿⣿⣿⣄⠀⠀
   ⢠⣿⣿⣿⣿⡿⠀⠈⠉⠀⠀⠀⠀⠉⠁⠀⢿⣿⣿⣿⣿⣷⠀
   ⣿⣿⣿⣿⣿⣇⠀⠀⠀⠀⡀⢀⠀⠀⠀⠀⣸⣿⣿⣿⣿⣿⡆
   ⠹⣿⣿⣿⣿⣿⠀⠀⠴⠞⠁⠈⠳⠦⠀⠀⣿⣿⣿⣿⣿⡿⠁
   ⠀⠉⢻⡿⢿⣿⣧⠀⠀⠀⢶⡶⠀⠀⠀⣼⣿⣿⣿⡟⠋⠁⠀
   ⠀⠀⣼⡇⠀⠀⠙⣷⣄⠀⠈⠁⠀⣠⣾⠋⠀⠀⢸⣧⠀⠀⠀
   ⠀⠀⣿⡇⠀⠀⠀⠈⠛⠷⣶⣶⠾⠛⠁⠀⠀⠀⢸⣿⠀⠀⠀
   ⠀⠀⢻⡇⠀⠀⠀⣀⣀⣤⣤⣤⣤⣀⣀⠀⠀⠀⢸⡟⠀⠀⠀
   ⠀⠀⠘⣿⣴⠾⠛⠋⠉⠉⠉⠉⠉⠉⠛⠛⠷⣦⣿⠃⠀⠀⠀
   ⠀⠀⠀⠈⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠁⠀⠀⠀⠀
All your word are belong to us!

     Shakespeare AI @ UI




Next we set our random seed, check to see if we have a CUDA-capable GPU.

If a GPU is detected, we use it.

In [38]:
#
# Set seed and device (GPU or CPU)
#
torch.manual_seed(SEED)
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print("DEVICE=", device)
print("\n")

DEVICE= cuda




Read the Complete Works of Shakespeare into a single string

From:  Project Gutenburg

https://www.gutenberg.org/ebooks/100.txt.utf-8

In [44]:

#print("Reading input...", flush=True)
#with open(INPUT_CORPUS, 'r', encoding='utf-8') as f:
#    text = f.read()

# Send a GET request to the URL
response = requests.get(INPUT_URL)

# Check if the request was successful
if response.status_code == 200:
    # Read the content as text
    text = response.text
else:
    print(f"Failed to fetch the file: HTTP {response.status_code}")
    exit(-1)

print("Hey we just read the complete works of Shakespeare directly from Project Gutenburg!")
print("Length = %d characters" %len(text))


vocab      = list(set(text))
vocab_size = len(vocab)
print("Detected vocabulary of size %d in corpus." %vocab_size, flush=True)


Hey we just read the complete works of Shakespeare directly from Project Gutenburg!
Length = 5575084 characters
Detected vocabulary of size 108 in corpus.


Our simple lookup-table tokenizer for characters.

In [46]:
#
# Tokenizer
#
char_to_idx  = { ch:i for i,ch in enumerate(vocab) }
idx_to_char  = { i:ch for i,ch in enumerate(vocab) }
string2tokens = lambda string: [char_to_idx[character] for character in string]
tokens2string = lambda tokens: ''.join([idx_to_char[index] for index in tokens])

tokenized_corpus = torch.tensor(string2tokens(text), dtype=torch.long)

Derive our TEST and VALIDATION sets from the tokenized corpus

In [47]:

train_portion = int(TRAIN_SIZE*len(tokenized_corpus))
training_data   = tokenized_corpus[:train_portion]
validation_data = tokenized_corpus[train_portion:]

Extract a batch of size BATCH_SIZE from either training or validation sets

In [48]:

def batch(data):
    selected = torch.randint(len(data) - CONTEXT_WINDOW, (BATCH_SIZE,))
    input    = torch.stack([data[i:i+CONTEXT_WINDOW] for i in selected])
    target   = torch.stack([data[i+1:i+CONTEXT_WINDOW+1] for i in selected])

    return input.to(device), target.to(device)

Evaluate the model during training to estimate average loss for the given dataset (training or validation)

In [49]:

def sample_loss(data):

    loss_tensor = torch.zeros(LOSS_SAMPLE_SIZE)
    for i in range(LOSS_SAMPLE_SIZE):
        input, target = batch(data)
        _,loss = model(input, target)
        loss_tensor[i] = loss.item()

    model.train()
    return(loss_tensor.mean())

ATTENTION HEAD definition - Here is where the magic happens!

In [50]:

class AttentionHead(nn.Module):

    def __init__(self, headspace):
        super().__init__()

        # The Query(Q), Key(K), and Value(V) vectors are just linear tensors
        self.query = nn.Linear(EMBEDDING_SIZE, headspace)
        self.key   = nn.Linear(EMBEDDING_SIZE, headspace)
        self.value = nn.Linear(EMBEDDING_SIZE, headspace)

        # Causal Self-Attention:  define our triangular mask so we can't look into the FUTURE...
        lower_diag = torch.tril(torch.ones(CONTEXT_WINDOW, CONTEXT_WINDOW))
        self.register_buffer('causal_mask', lower_diag)


    def forward(self, x):
        batchsize, seqlen, embedlen  = x.shape
        query_vector = self.query(x)
        key_vector   = self.key(x)

        attention_weights = query_vector @ key_vector.transpose(-2,-1) * torch.sqrt(torch.tensor(embedlen))
        attention_weights = attention_weights.masked_fill(self.causal_mask[:seqlen, :seqlen] == 0, float('-inf'))
        attention_weights = F.softmax(attention_weights, dim=-1)

        # weighted sum of value vector
        value_vector = self.value(x)
        x = attention_weights @ value_vector

        return x


MULTI-HEAD ATTENTION - Just a bunch of heads paying attention to different things.

In [51]:

class MultiHeadAttention(nn.Module):

    def __init__(self):
        super().__init__()

        # define NUM_HEADS attention heads, each head able to attend to only an equal-sized subset of the embedding layer
        headspace = EMBEDDING_SIZE // NUM_HEADS
        self.heads  = nn.ModuleList([AttentionHead(headspace) for h in range(NUM_HEADS)])
        self.linear = nn.Linear(EMBEDDING_SIZE, EMBEDDING_SIZE)

    def forward(self, x):

        # concatenate the output from all heads and aggregate through a single linear layer
        x = torch.cat([head(x) for head in self.heads], dim=-1)
        x = self.linear(x)

        return x


Define FEED FORWARD NETWORK (FFN) portion of the TransformerBlock

In [53]:

class TransformerFeedFoward(nn.Module):

    def __init__(self):

        super().__init__()

        self.feed_forward = nn.Sequential(
            nn.Linear(EMBEDDING_SIZE, EMBEDDING_SIZE*FFN_HIDDEN_SIZE),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Linear(EMBEDDING_SIZE*FFN_HIDDEN_SIZE, EMBEDDING_SIZE),
        )

    def forward(self, x):
        x = self.feed_forward(x)

        return(x)



Define the TRANSFORMER BLOCK component, consisting of:

    * Multiple Self-Attention Heads
    * Feed-Forward neural networks
    * Repeating Linear Layer Normalization layers


In [54]:

class TransformerBlock(nn.Module):

    def __init__(self):
        super().__init__()
        self.self_attention = MultiHeadAttention()
        self.feed_forward   = TransformerFeedFoward()
        self.linear_norm    = nn.LayerNorm(EMBEDDING_SIZE)

    def forward(self, x):

        lnorm0 = self.linear_norm(x)
        x = x + self.self_attention(lnorm0)

        lnorm1 = self.linear_norm(x)
        x = x + self.feed_forward(lnorm1)

        return x


Here we define our overall transformer called "ShakeGPT".  

Our transformer consists of:
  1. Embedding Layer
  2. Positional Encoding Layer
  3. A sequence of Transformer Blocks
  4. Layer Normalization layer
  5. Linear Layer

Loss is calculated using Cross Entropy.

We also provide a method for inference from the trained model within the class.


In [55]:
class ShakeGPT(nn.Module):

    def __init__(self):

        super().__init__()

        self.token_embedding_table    = nn.Embedding(vocab_size, EMBEDDING_SIZE)
        self.position_embedding_table = nn.Embedding(CONTEXT_WINDOW, EMBEDDING_SIZE)
        self.blocks                   = nn.Sequential(*[TransformerBlock() for _ in range(NUM_LAYERS)])
        self.layernorm                = nn.LayerNorm(EMBEDDING_SIZE) # final layer norm
        self.lm_head                  = nn.Linear(EMBEDDING_SIZE, vocab_size)

    def forward(self, input, targets=None):
        batchsize, seqlen = input.shape

        tok_emb = self.token_embedding_table(input)
        pos_emb = self.position_embedding_table(torch.arange(seqlen, device=device))
        x       = tok_emb + pos_emb
        x       = self.blocks(x)
        x       = self.layernorm(x)
        logits  = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            batchsize, seqlen, embedlen = logits.shape
            logits  = logits.view(batchsize*seqlen, embedlen)
            targets = targets.view(batchsize*seqlen)
            loss    = F.cross_entropy(logits, targets)

        return logits, loss


    def generate(self, idx, length):
        for i in range(length):
            idxc         = idx[:, -CONTEXT_WINDOW:]
            logits,_     = self(idxc)
            logits       = logits[:, -1, :]
            probs        = F.softmax(logits, dim=-1)
            idx_next     = torch.multinomial(probs, num_samples=1)
            idx          = torch.cat((idx, idx_next), dim=1)

        return idx

Instantiate our PyTorch transformer model and send it to the GPU

In [56]:

model     = ShakeGPT().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)


*** TRAINING ***

Train in batches of size BATCH_SIZE for NUM_BATCHES

Occasionally evaluate the current model to determine training and validation loss


In [57]:

for b in range(NUM_BATCHES):

    # Pull a random training batch
    input, target = batch(training_data)

    # Forward pass of data through the model
    _,loss = model(input, target)

    optimizer.zero_grad() # Reset our gradients for this batch
    loss.backward()       # Backpropagation
    optimizer.step()      # Neural Network Optimization

    # Every 100 batches, let's sample and report our loss
    if(not b%100):
        torch.no_grad()
        model.eval()
        train_loss = sample_loss(training_data)
        val_loss   = sample_loss(validation_data)
        print(f"BATCH %s/%d: training loss=%.05f | validation loss=%.05f" %(str(b).zfill(4),NUM_BATCHES,train_loss,val_loss))
        model.train()



BATCH 0000/2000: training loss=4.67348 | validation loss=4.66760
BATCH 0100/2000: training loss=2.82680 | validation loss=2.83257
BATCH 0200/2000: training loss=2.51810 | validation loss=2.54301
BATCH 0300/2000: training loss=2.37529 | validation loss=2.40609
BATCH 0400/2000: training loss=2.28504 | validation loss=2.31479
BATCH 0500/2000: training loss=2.20793 | validation loss=2.24271
BATCH 0600/2000: training loss=2.13698 | validation loss=2.17996
BATCH 0700/2000: training loss=2.08139 | validation loss=2.12036
BATCH 0800/2000: training loss=2.03329 | validation loss=2.07470
BATCH 0900/2000: training loss=1.99067 | validation loss=2.03509
BATCH 1000/2000: training loss=1.96226 | validation loss=2.00280
BATCH 1100/2000: training loss=1.92394 | validation loss=1.97502
BATCH 1200/2000: training loss=1.90801 | validation loss=1.95299
BATCH 1300/2000: training loss=1.87386 | validation loss=1.92639
BATCH 1400/2000: training loss=1.85407 | validation loss=1.90607
BATCH 1500/2000: training

*** INFERENCE ***

Training done.  Let's spew a chunk of Shakespearish!

In [58]:

print("INFERENCE TIME.  SPEWING...(be patient)", flush=True)

f = open(OUTPUT_FILE, "w", encoding="utf-8")
context = torch.zeros((1, 1), dtype=torch.long, device=device)
x = model.generate(context, length=5000)
generated_text = tokens2string(x[0].tolist())
print(generated_text)
f.write(generated_text)
f.close()

INFERENCE TIME.  SPEWING...(be patient)

LANTER.
Cack foul I would for swan windryen for man mories they call, I sives anfer laught knand.
Gone, ight in your hones a us fell?
Heing, that is that but odung-geis
 Edgaver of your and lay he srong coorn?
Nem.
I will not and they jera woe well;
O dath wouth, he brook it with him she rabout thou lef
FIk it twelfor miner that but of prearit
Traster of my thus, giul, which are—Thant
So Jownly, and Curcarr’d, that at the not cat
ancelfits is ony your greson, head
bechion, there this were dignotume
Way will hear? And ost wcom ditus more of good not was for entrute bod, seecuobs fir’stim not bed a dame owring thy home, upon men you of Bultom! I prate, canscrosed,
        That to you falEer TITON.
’F this will talk of with yecamer
And to bomen tu-grast ople leading’s of’ think aât macs’d
And untogs doneshs were ulash, for oton, Lady?

P(APALLO.
He that I will my an her make I have dets and awibmaw?

KINT INoth your for or spreport in poorbferice, 

# DONE!